In [150]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import json
import ast

### 1. annotation 가져오기
Get bbox location for image cropping

In [151]:
# JSON 파일 경로
json_file_path = 'dataset/person_keypoints_val2017.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    data = json.load(f)

In [152]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [153]:
data['annotations'][0].keys()

dict_keys(['segmentation', 'num_keypoints', 'area', 'iscrowd', 'keypoints', 'image_id', 'bbox', 'category_id', 'id'])

In [154]:
image_id = []
bbox = []
id = []
gt_keypoints = []

for item in data['annotations']:
    image_id.append(item['image_id'])
    bbox.append(item['bbox'])
    id.append(item['id'])
    gt_keypoints.append(item['keypoints'])

In [155]:
gt_df = pd.DataFrame({'image_id':image_id, 'id': id, 'bbox':bbox, 'gt_keypoints':gt_keypoints})
gt_df

,image_id,id,bbox,gt_keypoints
0,425226,183126,"[73.35, 206.02, 300.58, 372.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,440475,183302,"[491.91, 183.51, 34.85, 73.4]","[508, 192, 2, 510, 191, 2, 506, 191, 2, 512, 1..."
2,266400,183830,"[36.32, 0.59, 38.56, 53.86]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,69213,184249,"[276.35, 9.6, 244.69, 400.13]","[389, 97, 2, 405, 82, 2, 376, 80, 2, 434, 85, ..."
4,555050,184327,"[97.1, 185.73, 28.66, 27.72]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
10999,259690,900100259690,"[27, 98, 225, 114]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11000,145020,900100145020,"[27, 201, 209, 68]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11001,562818,900100562818,"[353, 229, 180, 39]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11002,439994,900100439994,"[0, 0, 427, 458]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [156]:
gt_df["width"] = gt_df["bbox"].apply(lambda bbox: bbox[2])
gt_df["height"] = gt_df["bbox"].apply(lambda bbox: bbox[3])
gt_df["bbox_size"] = gt_df["width"]*gt_df["height"]

In [157]:
gt_df

,image_id,id,bbox,gt_keypoints,width,height,bbox_size
0,425226,183126,"[73.35, 206.02, 300.58, 372.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",300.58,372.50,111966.0500
1,440475,183302,"[491.91, 183.51, 34.85, 73.4]","[508, 192, 2, 510, 191, 2, 506, 191, 2, 512, 1...",34.85,73.40,2557.9900
2,266400,183830,"[36.32, 0.59, 38.56, 53.86]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",38.56,53.86,2076.8416
3,69213,184249,"[276.35, 9.6, 244.69, 400.13]","[389, 97, 2, 405, 82, 2, 376, 80, 2, 434, 85, ...",244.69,400.13,97907.8097
4,555050,184327,"[97.1, 185.73, 28.66, 27.72]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",28.66,27.72,794.4552
...,...,...,...,...,...,...,...
10999,259690,900100259690,"[27, 98, 225, 114]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",225.00,114.00,25650.0000
11000,145020,900100145020,"[27, 201, 209, 68]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",209.00,68.00,14212.0000
11001,562818,900100562818,"[353, 229, 180, 39]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",180.00,39.00,7020.0000
11002,439994,900100439994,"[0, 0, 427, 458]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",427.00,458.00,195566.0000


### 2. model result에서 score 가져오기

In [158]:
# JSON 파일 경로
json_file_path = 'image_processing/result_keypoints_valid.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    result = json.load(f)

In [159]:
result[0].keys()

dict_keys(['bbox_id', 'category_id', 'center', 'image_id', 'keypoints', 'scale', 'score'])

In [160]:
len(result)

6332

In [161]:
center = []
image_id = []
scale = []
score = []
bbox_id = []

for item in result:
    center.append(item['center'])
    image_id.append(item['image_id'])
    scale.append(item['scale'])
    score.append(item['score'])
    bbox_id.append(item['bbox_id'])

In [162]:
result_df = pd.DataFrame({'image_id':image_id,'center':center,'scale':scale,'score':score, 'bbox_id':bbox_id})
result_df

,image_id,center,scale,score,bbox_id
0,397133,"[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",0.884479,0
1,397133,"[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",0.613978,1
2,252219,"[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",0.908204,1
3,252219,"[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",0.898217,0
4,252219,"[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",0.819430,2
...,...,...,...,...,...
6327,15335,"[502.4599914550781, 243.80499267578125]","[2.14439058303833, 2.859187364578247]",0.646088,0
6328,15335,"[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",0.594427,6
6329,15335,"[578.4849853515625, 91.56999969482422]","[0.6596249938011169, 0.8794999718666077]",0.571042,5
6330,15335,"[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",0.472550,4


- image_id -> score 정렬되어있음

In [163]:
# # 데이터를 평평한 형태로 변환
# flat_data = [(image_id, bbox_data) for image_id, bbox_data_list in result_df.items() for bbox_data in bbox_data_list]

# # 데이터프레임으로 변환
# result_df = pd.DataFrame(flat_data, columns=['image_id', 'bbox_data'])

# # JSON normalization을 통해 nested된 데이터를 펼침
# df_normalized = pd.json_normalize(result_df['bbox_data'])

# # 기존 'bbox_data' 열 제거 및 새로운 열로 추가
# result_df = pd.concat([result_df.drop(['bbox_data'], axis=1), df_normalized], axis=1)

# # 결과 확인
# print(result_df)

In [164]:
result_df.columns

Index(['image_id', 'center', 'scale', 'score', 'bbox_id'], dtype='object')

In [165]:
# tmp = result_df[['image_id', 'center', 'scale', 'score']]
# result_df = tmp.iloc[:,1:]
# result_df.columns

In [166]:
# bbox에서 kpts 저장되어있는 모델 중간 json 파일
# JSON 파일 경로
json_file_path = 'dataset/GT_result.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    kpt_result = json.load(f)

In [167]:
kpt_result['397133'][0].keys()

dict_keys(['keypoints', 'center', 'scale', 'area', 'score', 'image_id', 'bbox_id'])

In [168]:
# 데이터프레임을 담을 리스트 생성
df_list = []

# JSON 데이터를 데이터프레임으로 변환
for image_id, bbox_list in kpt_result.items():
    for bbox_id, bbox_info in enumerate(bbox_list):
        # 필요한 정보 추출
        bbox_data = {
            "image_id": int(image_id),
            "bbox_id": bbox_id,
            "keypoints": bbox_info["keypoints"]["__ndarray__"],
            "center": bbox_info["center"]["__ndarray__"],
            "scale": bbox_info["scale"]["__ndarray__"],
            "score": bbox_info["score"],
        }
        df_list.append(bbox_data)

# 리스트를 데이터프레임으로 변환
kpt_df = pd.DataFrame(df_list)

In [169]:
print(kpt_df.columns)
print(len(kpt_df))

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score'], dtype='object')
6352


In [170]:
kpt_df.sort_values(by='image_id')

,image_id,bbox_id,keypoints,center,scale,score
5249,139,0,"[[429.280517578125, 169.61883544921875, 0.8657...","[438.82501220703125, 226.11500549316406]","[0.6422343850135803, 0.8563125133514404]",1.0
261,785,0,"[[367.1922302246094, 80.87351989746094, 0.9483...","[389.6400146484375, 217.57000732421875]","[1.6203750371932983, 2.1605000495910645]",1.0
1487,872,0,"[[214.49267578125, 184.88751220703125, 0.79049...","[290.7349853515625, 328.8450012207031]","[2.1391406059265137, 2.852187395095825]",1.0
1488,872,1,"[[215.4751434326172, 182.15756225585938, 0.751...","[296.07501220703125, 366.1199951171875]","[2.247187614440918, 2.9962501525878906]",1.0
946,885,1,"[[378.148681640625, 110.68104553222656, 0.9212...","[336.43499755859375, 174.0850067138672]","[0.7900781035423279, 1.0534374713897705]",1.0
...,...,...,...,...,...,...
188,580294,0,"[[587.7160034179688, 73.6966552734375, 0.31208...","[525.0700073242188, 220.07000732421875]","[2.034562587738037, 2.712750196456909]",1.0
1769,581062,0,"[[163.4222869873047, 66.69705200195312, 0.9830...","[162.58999633789062, 107.5250015258789]","[0.5272969007492065, 0.7030624747276306]",1.0
2588,581206,0,"[[347.857177734375, 109.86737060546875, 0.1125...","[234.26499938964844, 109.66000366210938]","[2.928312301635742, 3.904416561126709]",1.0
3849,581317,0,"[[501.866455078125, 118.10910034179688, 0.9580...","[511.06500244140625, 204.46499633789062]","[1.3925156593322754, 1.8566875457763672]",1.0


In [171]:
print(kpt_df.columns)
print(result_df.columns)
print(gt_df.columns)

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score'], dtype='object')
Index(['image_id', 'center', 'scale', 'score', 'bbox_id'], dtype='object')
Index(['image_id', 'id', 'bbox', 'gt_keypoints', 'width', 'height',
       'bbox_size'],
      dtype='object')


In [172]:
result_df['center']

0         [442.864990234375, 208.22999572753906]
1       [30.579999923706055, 280.69500732421875]
2         [70.26000213623047, 279.7850036621094]
3        [361.3999938964844, 272.68499755859375]
4           [571.77001953125, 278.6499938964844]
                          ...                   
6327     [502.4599914550781, 243.80499267578125]
6328     [426.94000244140625, 109.0199966430664]
6329      [578.4849853515625, 91.56999969482422]
6330       [533.0349731445312, 99.5199966430664]
6331     [198.88499450683594, 108.5250015258789]
Name: center, Length: 6332, dtype: object

In [173]:
kpt_df['center']

0         [442.864990234375, 208.22999572753906]
1       [30.579999923706055, 280.69500732421875]
2        [361.3999938964844, 272.68499755859375]
3         [70.26000213623047, 279.7850036621094]
4           [571.77001953125, 278.6499938964844]
                          ...                   
6347                 [107.06500244140625, 259.5]
6348       [533.0349731445312, 99.5199966430664]
6349      [578.4849853515625, 91.56999969482422]
6350     [426.94000244140625, 109.0199966430664]
6351     [198.88499450683594, 108.5250015258789]
Name: center, Length: 6352, dtype: object

In [174]:
kpt_df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score'], dtype='object')

In [175]:
result_df.columns

Index(['image_id', 'center', 'scale', 'score', 'bbox_id'], dtype='object')

In [176]:
# kpt_df와 result_df를 image_id와 bbox_id를 기준으로 병합
merged_df = pd.merge(kpt_df, result_df, on=['image_id', 'bbox_id'], how='inner', suffixes=('_df1', '_df2'))

# 결과 확인
print(merged_df)

      image_id  bbox_id                                          keypoints  \
0       397133        0  [[430.8645324707031, 95.42893981933594, 0.9389...   
1       397133        1  [[52.14830780029297, 284.97412109375, 0.023037...   
2       252219        0  [[354.296630859375, 198.06585693359375, 0.9260...   
3       252219        1  [[100.3197021484375, 189.2662811279297, 0.9693...   
4       252219        2  [[537.075927734375, 193.65164184570312, 0.9272...   
...        ...      ...                                                ...   
6327     15335        3  [[83.04756164550781, 175.28688049316406, 0.925...   
6328     15335        4  [[508.6524963378906, 30.947364807128906, 0.216...   
6329     15335        5  [[561.7669067382812, 92.91932678222656, 0.1978...   
6330     15335        6  [[408.19903564453125, 93.58609008789062, 0.499...   
6331     15335        7  [[150.113037109375, 94.42748260498047, 0.08892...   

                                    center_df1  \
0       [442.

In [177]:
# # 기존의 key 열은 삭제
# df1 = pd.concat([kpt_df, kpt_df_expanded], axis=1)
# df2 = pd.concat([result_df, result_df_expanded], axis=1)

# # 키를 기준으로 데이터프레임 합치기
# merged_df = pd.merge(df1, df2, how='inner', on=['center_x', 'center_y'], suffixes=('_df1', '_df2'))

In [178]:
merged_df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center_df1', 'scale_df1',
       'score_df1', 'center_df2', 'scale_df2', 'score_df2'],
      dtype='object')

In [179]:
merged_df[merged_df['score_df1'] != merged_df['score_df2']][['score_df1', 'score_df2']]

,score_df1,score_df2
0,1.0,0.884479
1,1.0,0.613978
2,1.0,0.898217
3,1.0,0.908204
4,1.0,0.819430
...,...,...
6327,1.0,0.729402
6328,1.0,0.472550
6329,1.0,0.571042
6330,1.0,0.594427


In [180]:
# 다른 값 있는지 확인 
print(merged_df[merged_df['center_df1'] != merged_df['center_df2']])
print(merged_df[merged_df['scale_df1'] != merged_df['scale_df2']])

Empty DataFrame
Columns: [image_id, bbox_id, keypoints, center_df1, scale_df1, score_df1, center_df2, scale_df2, score_df2]
Index: []
Empty DataFrame
Columns: [image_id, bbox_id, keypoints, center_df1, scale_df1, score_df1, center_df2, scale_df2, score_df2]
Index: []


In [181]:
# 중복열 제거
merged_df.drop(columns=['scale_df2', 'center_df2', 'score_df1'], inplace=True, axis=1)

In [182]:
merged_df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center_df1', 'scale_df1',
       'score_df2'],
      dtype='object')

In [183]:
merged_df.rename(columns={'center_df1':'center', 'score_df2':'score', 'scale_df1':'scale'}, inplace=True)
print(merged_df.shape)
print(merged_df.columns)

(6332, 6)
Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score'], dtype='object')


In [184]:
# 리스트를 풀어서 각각의 요소를 새로운 열로 만들기
expanded = merged_df['center'].apply(pd.Series).rename(columns={0:'center_x',1:'center_y'})
merged_df = pd.concat([merged_df, expanded], axis=1)

In [185]:
merged_df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score',
       'center_x', 'center_y'],
      dtype='object')

In [186]:
gt_df.columns

Index(['image_id', 'id', 'bbox', 'gt_keypoints', 'width', 'height',
       'bbox_size'],
      dtype='object')

In [187]:
# 기존의 _xywh2cs 함수 정의
def _xywh2cs(x, y, w, h, padding=1.25):
    aspect_ratio = 1.0  # 임의로 aspect_ratio를 1.0으로 설정
    center = np.array([x + w * 0.5, y + h * 0.5], dtype=np.float32)

    if np.random.rand() < 0.3:
        center += 0.4 * (np.random.rand(2) - 0.5) * [w, h]

    if w > aspect_ratio * h:
        h = w * 1.0 / aspect_ratio
    elif w < aspect_ratio * h:
        w = h * aspect_ratio

    scale = np.array([w / 200.0, h / 200.0], dtype=np.float32)
    scale = scale * padding

    return center, scale

In [188]:
# bbox 열을 기반으로 x, y, width, height 열을 생성
gt_df[['x', 'y', 'width', 'height']] = pd.DataFrame(gt_df['bbox'].tolist(), index=gt_df.index)

# _xywh2cs 함수를 이용하여 center, scale 계산
gt_df[['center', 'scale']] = gt_df.apply(lambda row: pd.Series(_xywh2cs(row['x'], row['y'], row['width'], row['height'])), axis=1)


print(gt_df)

       image_id            id                            bbox  \
0        425226        183126  [73.35, 206.02, 300.58, 372.5]   
1        440475        183302   [491.91, 183.51, 34.85, 73.4]   
2        266400        183830     [36.32, 0.59, 38.56, 53.86]   
3         69213        184249   [276.35, 9.6, 244.69, 400.13]   
4        555050        184327    [97.1, 185.73, 28.66, 27.72]   
...         ...           ...                             ...   
10999    259690  900100259690              [27, 98, 225, 114]   
11000    145020  900100145020              [27, 201, 209, 68]   
11001    562818  900100562818             [353, 229, 180, 39]   
11002    439994  900100439994                [0, 0, 427, 458]   
11003    250282  900100250282               [0, 34, 639, 388]   

                                            gt_keypoints   width  height  \
0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  300.58  372.50   
1      [508, 192, 2, 510, 191, 2, 506, 191, 2, 512, 1...   34.85   

In [189]:
gt_df.columns

Index(['image_id', 'id', 'bbox', 'gt_keypoints', 'width', 'height',
       'bbox_size', 'x', 'y', 'center', 'scale'],
      dtype='object')

In [190]:
# 리스트를 풀어서 각각의 요소를 새로운 열로 만들기
expanded = gt_df['center'].apply(pd.Series).rename(columns={0:'center_x',1:'center_y'})
gt_df = pd.concat([gt_df, expanded], axis=1)

In [191]:
gt_df

,image_id,id,bbox,gt_keypoints,width,height,bbox_size,x,y,center,scale,center_x,center_y
0,425226,183126,"[73.35, 206.02, 300.58, 372.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",300.58,372.50,111966.0500,73.35,206.02,"[274.67746, 322.56427]","[2.328125, 2.328125]",274.677460,322.564270
1,440475,183302,"[491.91, 183.51, 34.85, 73.4]","[508, 192, 2, 510, 191, 2, 506, 191, 2, 512, 1...",34.85,73.40,2557.9900,491.91,183.51,"[508.25992, 212.08075]","[0.45875, 0.45875]",508.259918,212.080750
2,266400,183830,"[36.32, 0.59, 38.56, 53.86]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",38.56,53.86,2076.8416,36.32,0.59,"[55.6, 27.52]","[0.336625, 0.336625]",55.599998,27.520000
3,69213,184249,"[276.35, 9.6, 244.69, 400.13]","[389, 97, 2, 405, 82, 2, 376, 80, 2, 434, 85, ...",244.69,400.13,97907.8097,276.35,9.60,"[398.695, 209.665]","[2.5008125, 2.5008125]",398.695007,209.664993
4,555050,184327,"[97.1, 185.73, 28.66, 27.72]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",28.66,27.72,794.4552,97.10,185.73,"[111.13688, 198.49765]","[0.179125, 0.179125]",111.136879,198.497650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10999,259690,900100259690,"[27, 98, 225, 114]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",225.00,114.00,25650.0000,27.00,98.00,"[139.5, 155.0]","[1.40625, 1.40625]",139.500000,155.000000
11000,145020,900100145020,"[27, 201, 209, 68]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",209.00,68.00,14212.0000,27.00,201.00,"[131.5, 235.0]","[1.30625, 1.30625]",131.500000,235.000000
11001,562818,900100562818,"[353, 229, 180, 39]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",180.00,39.00,7020.0000,353.00,229.00,"[443.0, 248.5]","[1.125, 1.125]",443.000000,248.500000
11002,439994,900100439994,"[0, 0, 427, 458]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",427.00,458.00,195566.0000,0.00,0.00,"[213.5, 229.0]","[2.8625, 2.8625]",213.500000,229.000000


In [192]:
merged_df

,image_id,bbox_id,keypoints,center,scale,score,center_x,center_y
0,397133,0,"[[430.8645324707031, 95.42893981933594, 0.9389...","[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",0.884479,442.864990,208.229996
1,397133,1,"[[52.14830780029297, 284.97412109375, 0.023037...","[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",0.613978,30.580000,280.695007
2,252219,0,"[[354.296630859375, 198.06585693359375, 0.9260...","[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",0.898217,361.399994,272.684998
3,252219,1,"[[100.3197021484375, 189.2662811279297, 0.9693...","[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",0.908204,70.260002,279.785004
4,252219,2,"[[537.075927734375, 193.65164184570312, 0.9272...","[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",0.819430,571.770020,278.649994
...,...,...,...,...,...,...,...,...
6327,15335,3,"[[83.04756164550781, 175.28688049316406, 0.925...","[107.06500244140625, 259.5]","[1.7588437795639038, 2.3451249599456787]",0.729402,107.065002,259.500000
6328,15335,4,"[[508.6524963378906, 30.947364807128906, 0.216...","[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",0.472550,533.034973,99.519997
6329,15335,5,"[[561.7669067382812, 92.91932678222656, 0.1978...","[578.4849853515625, 91.56999969482422]","[0.6596249938011169, 0.8794999718666077]",0.571042,578.484985,91.570000
6330,15335,6,"[[408.19903564453125, 93.58609008789062, 0.499...","[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",0.594427,426.940002,109.019997


In [198]:
from scipy.spatial.distance import cdist

def find_closest_gt(row, gt_df):
    if not pd.isna(row['center']).any():
        center = row['center']
        gt_for_image = gt_df[gt_df['image_id'] == row['image_id']]
        distances = cdist([center], gt_for_image['center'].tolist())
        closest_index = distances.argmin()
        closest_gt_row = gt_for_image.iloc[closest_index]
        return closest_gt_row
    else:
        return pd.Series([pd.NA] * len(gt_df.columns))

# merged_df의 각 행에 대해 가장 가까운 gt_df의 값을 찾아서 병합
df = pd.concat([merged_df, merged_df.apply(find_closest_gt, gt_df=gt_df, axis=1)], axis=1)

In [199]:
df

,image_id,bbox_id,keypoints,center,scale,score,center_x,center_y,image_id,id,...,gt_keypoints,width,height,bbox_size,x,y,center,scale,center_x,center_y
0,397133,0,"[[430.8645324707031, 95.42893981933594, 0.9389...","[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",0.884479,442.864990,208.229996,397133,200887,...,"[433, 94, 2, 434, 90, 2, 0, 0, 0, 443, 98, 2, ...",109.41,277.62,30374.4042,388.66,69.92,"[443.365, 208.73]","[1.7351251, 1.7351251]",443.364990,208.729996
1,397133,1,"[[52.14830780029297, 284.97412109375, 0.023037...","[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",0.613978,30.580000,280.695007,397133,1218137,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",62.16,36.77,2285.6232,0.00,262.81,"[22.034864, 278.0575]","[0.38849998, 0.38849998]",22.034864,278.057495
2,252219,0,"[[354.296630859375, 198.06585693359375, 0.9260...","[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",0.898217,361.399994,272.684998,252219,481918,...,"[356, 198, 2, 358, 193, 2, 351, 194, 2, 364, 1...",71.24,197.25,14052.0900,326.28,174.56,"[349.879, 235.91711]","[1.2328125, 1.2328125]",349.878998,235.917114
3,252219,1,"[[100.3197021484375, 189.2662811279297, 0.9693...","[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",0.908204,70.260002,279.785004,252219,489768,...,"[100, 190, 2, 0, 0, 0, 96, 185, 2, 0, 0, 0, 86...",121.94,226.45,27613.3130,9.79,167.06,"[70.76, 280.285]","[1.4153124, 1.4153124]",70.760002,280.285004
4,252219,2,"[[537.075927734375, 193.65164184570312, 0.9272...","[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",0.819430,571.770020,278.649994,252219,495624,...,"[536, 192, 1, 538, 188, 2, 0, 0, 0, 552, 190, ...",123.66,215.76,26680.8816,510.44,171.27,"[572.27, 279.15]","[1.3485, 1.3485]",572.270020,279.149994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6327,15335,3,"[[83.04756164550781, 175.28688049316406, 0.925...","[107.06500244140625, 259.5]","[1.7588437795639038, 2.3451249599456787]",0.729402,107.065002,259.500000,15335,1277736,...,"[84, 173, 2, 102, 162, 2, 71, 162, 2, 0, 0, 0,...",212.97,376.22,80123.5734,1.08,71.89,"[107.565, 260.0]","[2.351375, 2.351375]",107.565002,260.000000
6328,15335,4,"[[508.6524963378906, 30.947364807128906, 0.216...","[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",0.472550,533.034973,99.519997,15335,1282318,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 553, 42, ...",49.41,129.28,6387.7248,508.83,35.38,"[533.535, 100.02]","[0.80799997, 0.80799997]",533.534973,100.019997
6329,15335,5,"[[561.7669067382812, 92.91932678222656, 0.1978...","[578.4849853515625, 91.56999969482422]","[0.6596249938011169, 0.8794999718666077]",0.571042,578.484985,91.570000,15335,1694331,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",75.07,141.72,10638.9204,541.45,21.21,"[576.5006, 95.48112]","[0.88575, 0.88575]",576.500610,95.481117
6330,15335,6,"[[408.19903564453125, 93.58609008789062, 0.499...","[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",0.594427,426.940002,109.019997,15335,1705756,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 386, 88, 2, 444, 8...",167.62,111.74,18729.8588,343.63,53.65,"[427.44, 109.52]","[1.0476251, 1.0476251]",427.440002,109.519997


In [202]:
df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score',
       'center_x', 'center_y', 'image_id', 'id', 'bbox', 'gt_keypoints',
       'width', 'height', 'bbox_size', 'x', 'y', 'center', 'scale', 'center_x',
       'center_y'],
      dtype='object')

In [203]:
# 중복된 열 이름 확인
duplicated_columns = df.columns[df.columns.duplicated()].tolist()

# 이름이 같은 열 중 첫 번째 열만 남기고 나머지는 제거
filtered_df = df.loc[:, ~df.columns.duplicated()]

In [204]:
filtered_df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'score',
       'center_x', 'center_y', 'id', 'bbox', 'gt_keypoints', 'width', 'height',
       'bbox_size', 'x', 'y'],
      dtype='object')

In [205]:
filtered_df.to_csv("merged_df.csv", index=False)